In [1]:
# Dependencies
import pandas as pd
import nfl_data_py as nfl
import stat_collection as stats


# Display maximum columns
pd.set_option('display.max_columns', None)
#stats.to_csv('{}-{}playerstats.csv'.format(start_year, final_year)) # csv output format {year}playerstats.csv

In [2]:
# Set year of desired dataset
year = 2023


In [3]:
pass_df, def_df = stats.get_stats(year, 3)


2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.
